In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

C:\Users\elrii\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
input_dir = r'./input/'
output_dir = r'./output/'

In [109]:
dfa = pd.read_sas(input_dir+'AUX_1999.XPT')
dfq = pd.read_sas(input_dir+'AUQ_1999.XPT')
dfd = pd.read_sas(input_dir+'DEMO_1999.XPT') #CHANGE

C:\Users\elrii\AppData\Local\Temp\ipykernel_2560\1387263585.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dfa = pd.read_sas(input_dir+'AUX_1999.XPT')
C:\Users\elrii\AppData\Local\Temp\ipykernel_2560\1387263585.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dfa = pd.read_sas(input_dir+'AUX_1999.XPT')
C:\Users\elrii\AppData\Local\Temp\ipykernel_2560\1387263585.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider 

C:\Users\elrii\AppData\Local\Temp\ipykernel_2560\1387263585.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dfd = pd.read_sas(input_dir+'DEMO_1999.XPT') #CHANGE
C:\Users\elrii\AppData\Local\Temp\ipykernel_2560\1387263585.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dfd = pd.read_sas(input_dir+'DEMO_1999.XPT') #CHANGE
C:\Users\elrii\AppData\Local\Temp\ipykernel_2560\1387263585.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perfo

C:\Users\elrii\AppData\Local\Temp\ipykernel_2560\1387263585.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dfd = pd.read_sas(input_dir+'DEMO_1999.XPT') #CHANGE
C:\Users\elrii\AppData\Local\Temp\ipykernel_2560\1387263585.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dfd = pd.read_sas(input_dir+'DEMO_1999.XPT') #CHANGE
C:\Users\elrii\AppData\Local\Temp\ipykernel_2560\1387263585.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perfo

In [110]:
#dfq.to_excel(output_dir+"hi.xlsx")

In [111]:
dfa = dfa[['SEQN', 'AUXU500R','AUXU1K2R','AUXU2KR','AUXU3KR',
            'AUXU4KR','AUXU6KR','AUXU8KR','AUXU500L',
            'AUXU1K2L','AUXU2KL','AUXU3KL','AUXU4KL','AUXU6KL',
            'AUXU8KL']].replace(666,100)

audiogram_col = ['AUXU500R','AUXU1K2R','AUXU2KR','AUXU3KR',
            'AUXU4KR','AUXU6KR','AUXU8KR','AUXU500L',
            'AUXU1K2L','AUXU2KL','AUXU3KL','AUXU4KL','AUXU6KL',
            'AUXU8KL']
audiogram_right = audiogram_col[:7]
audiogram_left = audiogram_col[7:]

dfa = dfa[~dfa[audiogram_col].isin([888]).any(axis=1)].dropna().astype(int)
dfa = dfa[(dfa[audiogram_right].sum(axis=1) >= 140) & (dfa[audiogram_right].sum(axis=1) >= 140)].reset_index(drop=True)
dfa

,SEQN,AUXU500R,AUXU1K2R,AUXU2KR,AUXU3KR,AUXU4KR,AUXU6KR,AUXU8KR,AUXU500L,AUXU1K2L,AUXU2KL,AUXU3KL,AUXU4KL,AUXU6KL,AUXU8KL
0,57,45,20,30,40,45,50,80,-5,5,15,10,5,10,25
1,126,15,15,30,40,40,25,10,15,10,20,25,20,35,25
2,130,25,15,5,25,80,100,100,30,30,45,70,100,100,100
3,150,10,35,65,70,90,100,100,15,25,35,35,55,55,55
4,162,15,20,50,75,80,75,90,20,25,40,65,75,85,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526,9932,20,25,10,35,20,35,40,15,20,15,30,25,50,60
527,9935,10,20,55,55,60,65,70,20,25,50,55,65,60,60
528,9938,5,10,15,25,75,95,90,5,15,20,15,25,30,20
529,9944,35,20,15,55,55,60,80,30,15,10,25,35,50,50


In [112]:
dfq = dfq[['SEQN','AUQ130', 'AUQ190', 'AUQ230']] #CHECK
dfq=dfq.dropna(how='any').reset_index(drop=True)
dfq.shape

(4877, 4)

In [113]:
dfd = dfd[['SEQN', 'RIAGENDR','RIDAGEYR', 'DMQMILIT']] #1 = male, 2 = female

thresholds = [0, 25, 35, 45, 55, dfd['RIDAGEYR'].max()] 
labels = [1, 2, 3,4,5]
dfd['RIDAGEYR'] = pd.cut(dfd['RIDAGEYR'], 
                         bins=thresholds, labels=labels, right=True)
dfd = dfd.dropna(subset=['DMQMILIT']).reset_index(drop=True)
dfd.shape

C:\Users\elrii\AppData\Local\Temp\ipykernel_2560\1334525107.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfd['RIDAGEYR'] = pd.cut(dfd['RIDAGEYR'],


(5731, 4)

In [114]:
dfV = pd.merge(pd.merge(dfd, dfa, on='SEQN'), dfq).astype(int)
print(dfV.shape)
dfV.to_excel(output_dir+"NHANES_1999-2000.xlsx") #CHANGE

(531, 21)


In [115]:
dfV

,SEQN,RIAGENDR,RIDAGEYR,DMQMILIT,AUXU500R,AUXU1K2R,AUXU2KR,AUXU3KR,AUXU4KR,AUXU6KR,...,AUXU500L,AUXU1K2L,AUXU2KL,AUXU3KL,AUXU4KL,AUXU6KL,AUXU8KL,AUQ130,AUQ190,AUQ230
0,57,1,3,2,45,20,30,40,45,50,...,-5,5,15,10,5,10,25,1,2,2
1,126,2,2,2,15,15,30,40,40,25,...,15,10,20,25,20,35,25,1,1,1
2,130,1,5,2,25,15,5,25,80,100,...,30,30,45,70,100,100,100,2,1,1
3,150,1,5,2,10,35,65,70,90,100,...,15,25,35,35,55,55,55,2,2,2
4,162,1,5,1,15,20,50,75,80,75,...,20,25,40,65,75,85,80,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526,9932,2,4,2,20,25,10,35,20,35,...,15,20,15,30,25,50,60,1,2,2
527,9935,1,5,2,10,20,55,55,60,65,...,20,25,50,55,65,60,60,3,1,2
528,9938,1,4,1,5,10,15,25,75,95,...,5,15,20,15,25,30,20,2,2,1
529,9944,1,4,2,35,20,15,55,55,60,...,30,15,10,25,35,50,50,1,2,2
